In [19]:
import os
detals_filename ='names.txt'

details =open(detals_filename, mode="r", encoding="utf-8").readlines()

details[0]= "SOSNA"+"\n"
details[1]= "MEZENCWEVA"+"\n"
details[2] = "Vetupravlenie"+"\n"

str(details)

file= open(detals_filename, "w", encoding="utf-8")

str(details)
#file.write(str(details))

string_to_write = str()
string_to_write= string_to_write+details[0]
string_to_write= string_to_write+details[1]
string_to_write= string_to_write+details[2]
file.write(string_to_write)
file.close()

In [1]:
import sqlite3
import os

def merge_sqlite_databases(db1_path, db2_path, output_db_path, conflict_strategy='ignore'):
    """
    Объединяет две базы данных SQLite с одинаковой структурой таблиц
    в одну базу данных с обработкой конфликтов по первичным ключам.

    Parameters:
    db1_path (str): Путь к первой базе данных
    db2_path (str): Путь ко второй базе данных
    output_db_path (str): Путь для сохранения объединённой базы данных
    conflict_strategy (str): Стратегия обработки конфликтов ('ignore' или 'replace')
    """
    if conflict_strategy == 'ignore':
        insert_clause = 'INSERT OR IGNORE'
    elif conflict_strategy == 'replace':
        insert_clause = 'INSERT OR REPLACE'
    else:
        raise ValueError("conflict_strategy должен быть 'ignore' или 'replace'")

    # Копируем первую БД в выходной файл
    if os.path.exists(output_db_path):
        os.remove(output_db_path)
    with open(db1_path, 'rb') as f_src, open(output_db_path, 'wb') as f_dst:
        f_dst.write(f_src.read())

    conn_out = sqlite3.connect(output_db_path)
    cursor_out = conn_out.cursor()

    # Подключаем вторую базу как ATTACH
    cursor_out.execute(f"ATTACH DATABASE '{db2_path}' AS db2")

    # Получаем список таблиц в основной базе (output),
    # игнорируем служебные
    cursor_out.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'")
    tables = [row[0] for row in cursor_out.fetchall()]

    for table in tables:
        sql = f'{insert_clause} INTO {table} SELECT * FROM db2.{table}'
        try:
            cursor_out.execute(sql)
        except Exception as e:
            print(f"Ошибка при вставке данных в таблицу {table}: {e}")

    conn_out.commit()
    cursor_out.execute("DETACH DATABASE db2")
    conn_out.close()

    print(f"Объединение завершено. Итоговая база: {output_db_path}")

# Пример вызова:
# merge_sqlite_databases('db1.sqlite', 'db2.sqlite', 'merged_db.sqlite', conflict_strategy='replace')

In [6]:
db1_path='./nichego/baza1.sqlite3'
db2_path='./nichego/baza2.sqlite3'
output_db_path = 'result_database.sqlite3'
merge_sqlite_databases(db1_path, db2_path, output_db_path)

Ошибка вставки в таблицу gay_clubs: UNIQUE constraint failed: gay_clubs.pk
Ошибка вставки в таблицу porno: UNIQUE constraint failed: porno.pk
Объединение завершено. Итоговая база: result_database.sqlite3


In [9]:
import sqlite3
import pandas as pd
import base64
import os
import secrets

def load_dbs_as_dfs(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Получаем список таблиц
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'")
    tables = [row[0] for row in cursor.fetchall()]

    dfs = {}
    for table in tables:
        dfs[table] = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    conn.close()
    return dfs

def generate_256bit_base64():
    rand_bytes = secrets.token_bytes(32)  # 256 бит = 32 байта
    return base64.urlsafe_b64encode(rand_bytes).rstrip(b'=').decode('ascii')

def replace_keys_base64(dfs, pk_fk_info):
    """
    dfs: dict таблица -> dataframe
    pk_fk_info: dict с описанием, где первичные ключи и внешние ключи, вида:
      {
        'table_name': {
           'pk': 'primary_key_column_name',
           'fks': { 'fk_column_name': 'referenced_table' }
        },
        ...
      }

    Возвращает новый словарь датафреймов с заменёнными ключами в Base64.
    """
    # Словарь для хранения маппинга старых pk -> новых base64 ключей для каждой таблицы
    pk_maps = {}

    dfs_new = {}

    # Сначала для каждой таблицы заменим PK на base64 значения и запомним сопоставление
    for table, df in dfs.items():
        info = pk_fk_info.get(table, {})
        pk_col = info.get('pk')
        if pk_col is None:
            # Нет PK — просто копируем таблицу без изменений
            dfs_new[table] = df.copy()
            continue

        # Создаём мапинг
        old_pks = df[pk_col].tolist()
        new_pks = [generate_256bit_base64() for _ in old_pks]

        pk_map = dict(zip(old_pks, new_pks))
        pk_maps[table] = pk_map

        df_copy = df.copy()
        df_copy[pk_col] = df_copy[pk_col].map(pk_map)
        dfs_new[table] = df_copy

    # Теперь обновим внешние ключи, если они есть
    for table, df in dfs_new.items():
        info = pk_fk_info.get(table, {})
        fks = info.get('fks', {})
        if not fks:
            continue

        df_copy = df.copy()

        for fk_col, ref_table in fks.items():
            if fk_col not in df_copy.columns:
                continue
            if ref_table not in pk_maps:
                continue
            fk_map = pk_maps[ref_table]
            # Для значений внешнего ключа, которые есть в fk_map, меняем, а если нет — оставляем как есть
            df_copy[fk_col] = df_copy[fk_col].map(lambda x: fk_map.get(x, x))
        dfs_new[table] = df_copy

    return dfs_new

def merge_dfs(dfs1, dfs2):
    """
    Объединяем две словаря таблиц (dataframe) по ключу (названию таблиц)
    """
    merged = {}
    all_tables = set(dfs1.keys()).union(dfs2.keys())
    for table in all_tables:
        df1 = dfs1.get(table)
        df2 = dfs2.get(table)
        if df1 is not None and df2 is not None:
            merged_df = pd.concat([df1, df2], ignore_index=True)
        elif df1 is not None:
            merged_df = df1.copy()
        else:
            merged_df = df2.copy()
        merged[table] = merged_df
    return merged

def replace_base64_with_autoinc(dfs, pk_fk_info):
    """
    Заменяет base64 ключи в объединённом наборе dfs на autoincrement int ключи,
    поправив внешние ключи.

    Возвращает новый набор dfs.
    """
    pk_maps = {}  # table_name -> { old_base64_key: new_int_key }

    dfs_new = {}

    # Сначала для каждой таблицы создаём маппинг base64 pk -> int pk
    for table, df in dfs.items():
        info = pk_fk_info.get(table, {})
        pk_col = info.get('pk')
        if pk_col is None:
            dfs_new[table] = df.copy()
            continue

        old_pks = df[pk_col].tolist()
        # Создаем инт айдишки, нам надо уникальные:
        unique_old_pks = pd.Series(old_pks).unique()
        pk_map = dict(zip(unique_old_pks, range(1, len(unique_old_pks) + 1)))
        pk_maps[table] = pk_map

        # Заменяем pk в df
        df_copy = df.copy()
        df_copy[pk_col] = df_copy[pk_col].map(pk_map)
        dfs_new[table] = df_copy

    # Обновляем foreign key
    for table, df in dfs_new.items():
        info = pk_fk_info.get(table, {})
        fks = info.get('fks', {})
        if not fks:
            continue
        df_copy = df.copy()
        for fk_col, ref_table in fks.items():
            if fk_col not in df_copy.columns:
                continue
            if ref_table not in pk_maps:
                continue
            fk_map = pk_maps[ref_table]
            df_copy[fk_col] = df_copy[fk_col].map(lambda x: fk_map.get(x, x))
        dfs_new[table] = df_copy

    return dfs_new

def write_dfs_to_sqlite(dfs, pk_fk_info, output_db_path):
    """
    Записывает датафреймы в SQLite файл

    При этом для таблиц с PK ставит INTEGER PRIMARY KEY AUTOINCREMENT,
    для других просто создаёт таблицы.

    !! Внимание !! 
    Здесь упрощённая логика создания таблиц — для реальной сложной схемы 
    стоит использовать инспектор схемы или ORM

    Принимает:
      - dfs: dict table->DataFrame
      - pk_fk_info: dict с info по key-ам
      - output_db_path: файл для записи
    """
    if os.path.exists(output_db_path):
        os.remove(output_db_path)

    conn = sqlite3.connect(output_db_path)
    cursor = conn.cursor()

    for table, df in dfs.items():
        info = pk_fk_info.get(table, {})
        pk_col = info.get('pk')
        fks = info.get('fks', {})

        # Формируем схему
        columns = df.columns.tolist()
        col_defs = []

        for col in columns:
            dtype = df[col].dtype
            if pd.api.types.is_integer_dtype(dtype):
                col_type = 'INTEGER'
            elif pd.api.types.is_float_dtype(dtype):
                col_type = 'REAL'
            else:
                col_type = 'TEXT'

            if pk_col == col:
                col_def = f"{col} INTEGER PRIMARY KEY AUTOINCREMENT"
            else:
                col_def = col + ' ' + col_type
            col_defs.append(col_def)

        sql_create = f"CREATE TABLE {table} ({', '.join(col_defs)}"

        # Добавляем внешние ключи (если есть)
        if fks:
            fk_defs = []
            for fk_col, ref_table in fks.items():
                pk_ref_col = pk_fk_info[ref_table]['pk']
                fk_defs.append(f"FOREIGN KEY ({fk_col}) REFERENCES {ref_table}({pk_ref_col})")
            if fk_defs:
                sql_create += ', ' + ', '.join(fk_defs)

        sql_create += ');'

        cursor.execute(sql_create)

        # Вставляем данные
        placeholders = ', '.join(['?' for _ in columns])
        insert_sql = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({placeholders})"

        values = df.where(pd.notnull(df), None).values.tolist()
        cursor.executemany(insert_sql, values)

    conn.commit()
    conn.close()

def merge_sqlite_dbs_with_key_replacement(db1_path, db2_path, output_db_path, pk_fk_info):
    """
    Основная функция:
    1) Загружает обе базы в DF
    2) Заменяет PK в обеих базах на 256бит base64 ключи с обновлением FK
    3) Объединяет DF по таблицам
    4) Заменяет base64 ключи на int PK с обновлением FK
    5) Записывает в SQLite

    pk_fk_info: структура с info по первичным и внешним ключам

    Пример pk_fk_info:
    {
       'authors': {
           'pk': 'author_id',
           'fks': {}
       },
       'books': {
           'pk': 'book_id',
           'fks': {'author_id': 'authors'}
       }
    }
    """

    print("Загрузка баз...")
    dfs1 = load_dbs_as_dfs(db1_path)
    dfs2 = load_dbs_as_dfs(db2_path)

    print("Заменяем PK на base64 в первой базе...")
    dfs1_b64 = replace_keys_base64(dfs1, pk_fk_info)

    print("Заменяем PK на base64 во второй базе...")
    dfs2_b64 = replace_keys_base64(dfs2, pk_fk_info)

    print("Объединяем датафреймы...")
    merged_dfs = merge_dfs(dfs1_b64, dfs2_b64)

    print("Заменяем base64 PK на int PK с автоинкрементом...")
    final_dfs = replace_base64_with_autoinc(merged_dfs, pk_fk_info)

    print("Записываем в SQLite...")
    write_dfs_to_sqlite(final_dfs, pk_fk_info, output_db_path)

    print(f"Обработка завершена. Итоговая база: {output_db_path}")


# --- ПРИМЕР ВЫЗОВА ---
# Необходимо задать pk_fk_info для вашей схемы БД (пример ниже)
# pk_fk_info = {
#   'table1': {'pk': 'id', 'fks': {'parent_id': 'table1'}},
#   'table2': {'pk': 'id', 'fks': {'table1_id': 'table1'}},
# }

# merge_sqlite_dbs_with_key_replacement('db1.sqlite', 'db2.sqlite', 'merged.sqlite', pk_fk_info)


In [10]:
pk_fk_info = {
    'animal_docs': {
        'pk': 'pk',
        'fks': {
            'animal': 'animals_simple',
            'cattle': 'cattle'
        }
    },
    'animals_simple': {
        'pk': 'pk',
        'fks': {
            'belongs_to_household': 'household'
        }
    },
    'cattle': {
        'pk': 'pk',
        'fks': {
            'belongs_to_household': 'household'
        }
    },
    'city': {
        'pk': 'pk',
        'fks': {
            'belongs_to_settlement': 'settlement'
        }
    },
    'conducted_tests': {
        'pk': 'pk',
        'fks': {
            'animal': 'cattle'
        }
    },
    'household': {
        'pk': 'pk',
        'fks': {
            'belongs_to_city': 'city'
        }
    },
    'report_entries': {
        'pk': 'pk',
        'fks': {
            'belongs_to_report': 'reports',
            'household': 'household'
        }
    },
    'reports': {
        'pk': 'pk',
        'fks': {
            'city': 'city'
        }
    },
    'settlement': {
        'pk': 'pk',
        'fks': {}
    }
}


In [11]:
# PRIMER BLIN
pk_fk_info = {
    'gay_clubs': {
        'pk': 'pk',
        'fks': {}
    },
    'porno': {
        'pk': 'pk',
        'fks': {
            'gay_club': 'gay_clubs'
        }
    }
}

merge_sqlite_dbs_with_key_replacement(db1_path, db2_path, output_db_path, pk_fk_info)

Загрузка баз...
Заменяем PK на base64 в первой базе...
Заменяем PK на base64 во второй базе...
Объединяем датафреймы...
Заменяем base64 PK на int PK с автоинкрементом...
Записываем в SQLite...
Обработка завершена. Итоговая база: result_database.sqlite3
